In [197]:
from pymongo import MongoClient
import pymongo
from pprint import pprint
import pandas as pd
import json
import re
pymongo.version

'4.10.1'

### Set up database

#### Bind 'mongo' variable to MongoClient
#### Note: move to same directory as input json file

In [146]:
mongo = MongoClient()

#### Create a Database and Bind it to the 'db' variable

In [149]:
db = mongo['electionResultsByYear_State_DistrictDB']

#### Create a Collection and Bind It To 'electionResults' variable

In [152]:
electionResults = db['electionResults']

#### Read JSON File

In [155]:
with open('election_results_by_district.json', 'r') as f:
    data = json.load(f)

#### Inset Data Into Database

In [158]:
electionResults.insert_many(data)

InsertManyResult([ObjectId('677727fa1f5535c6a24ea98a'), ObjectId('677727fa1f5535c6a24ea98b'), ObjectId('677727fa1f5535c6a24ea98c'), ObjectId('677727fa1f5535c6a24ea98d'), ObjectId('677727fa1f5535c6a24ea98e'), ObjectId('677727fa1f5535c6a24ea98f'), ObjectId('677727fa1f5535c6a24ea990'), ObjectId('677727fa1f5535c6a24ea991'), ObjectId('677727fa1f5535c6a24ea992'), ObjectId('677727fa1f5535c6a24ea993'), ObjectId('677727fa1f5535c6a24ea994'), ObjectId('677727fa1f5535c6a24ea995'), ObjectId('677727fa1f5535c6a24ea996'), ObjectId('677727fa1f5535c6a24ea997'), ObjectId('677727fa1f5535c6a24ea998'), ObjectId('677727fa1f5535c6a24ea999'), ObjectId('677727fa1f5535c6a24ea99a'), ObjectId('677727fa1f5535c6a24ea99b'), ObjectId('677727fa1f5535c6a24ea99c'), ObjectId('677727fa1f5535c6a24ea99d'), ObjectId('677727fa1f5535c6a24ea99e'), ObjectId('677727fa1f5535c6a24ea99f'), ObjectId('677727fa1f5535c6a24ea9a0'), ObjectId('677727fa1f5535c6a24ea9a1'), ObjectId('677727fa1f5535c6a24ea9a2'), ObjectId('677727fa1f5535c6a24ea9

#### Confirm DB is as expected

In [161]:
print(db.list_collection_names())

['electionResults']


In [163]:
query = {}
documentCount = electionResults.count_documents(query)
print(f"Document count: {documentCount}")

Document count: 1614


#### Confirm one record is as expected
should find record for Illinois:

  {'Congressional_District': 'Congressional District 1',
 'District_dem_votes': 231912.0,
 'District_rep_votes': 78624.0,
 'State': 'Illinois',
 'Year': 2016,
 '_id': ObjectId('677707ae1f5535c6a24ea6cf'),
 'state_fips': '17'}

In [166]:
query = {"State":"Illinois"}
results = electionResults.find(query)
pprint(results[0])

{'Congressional_District': 'Congressional District 1',
 'District_dem_votes': 231912.0,
 'District_rep_votes': 78624.0,
 'State': 'Illinois',
 'Year': 2016,
 '_id': ObjectId('677707ae1f5535c6a24ea6cf'),
 'state_fips': '17'}


### Access Data Examples

#### - Move Database into Dataframe

In [170]:
allData = list(electionResults.find())
election_results_df = pd.DataFrame(allData)
election_results_df.head()

,_id,Year,State,state_fips,Congressional_District,District_dem_votes,District_rep_votes
0,677707ae1f5535c6a24ea662,2016,Alabama,01,Congressional District 1,103967.0,192270.0
1,677707ae1f5535c6a24ea663,2016,Alabama,01,Congressional District 2,98925.0,183652.0
2,677707ae1f5535c6a24ea664,2016,Alabama,01,Congressional District 3,95117.0,183965.0
3,677707ae1f5535c6a24ea665,2016,Alabama,01,Congressional District 4,53969.0,225667.0
4,677707ae1f5535c6a24ea666,2016,Alabama,01,Congressional District 5,97169.0,200644.0


#### -Remove mongo object id

In [173]:
election_results_df = election_results_df.drop(['_id'], axis=1)
election_results_df.head()

,Year,State,state_fips,Congressional_District,District_dem_votes,District_rep_votes
0,2016,Alabama,01,Congressional District 1,103967.0,192270.0
1,2016,Alabama,01,Congressional District 2,98925.0,183652.0
2,2016,Alabama,01,Congressional District 3,95117.0,183965.0
3,2016,Alabama,01,Congressional District 4,53969.0,225667.0
4,2016,Alabama,01,Congressional District 5,97169.0,200644.0


#### - One All Records For One State By Name (with .find)

In [176]:
query = {"State":"Illinois"}
results = electionResults.find(query)
pprint(results[0])
i = 0
for result in results:
    if i < 3:
        pprint(result)
    i+=1

{'Congressional_District': 'Congressional District 1',
 'District_dem_votes': 231912.0,
 'District_rep_votes': 78624.0,
 'State': 'Illinois',
 'Year': 2016,
 '_id': ObjectId('677707ae1f5535c6a24ea6cf'),
 'state_fips': '17'}
{'Congressional_District': 'Congressional District 1',
 'District_dem_votes': 231912.0,
 'District_rep_votes': 78624.0,
 'State': 'Illinois',
 'Year': 2016,
 '_id': ObjectId('677707ae1f5535c6a24ea6cf'),
 'state_fips': '17'}
{'Congressional_District': 'Congressional District 10',
 'District_dem_votes': 176677.0,
 'District_rep_votes': 95923.0,
 'State': 'Illinois',
 'Year': 2016,
 '_id': ObjectId('677707ae1f5535c6a24ea6d0'),
 'state_fips': '17'}
{'Congressional_District': 'Congressional District 11',
 'District_dem_votes': 151614.0,
 'District_rep_votes': 124352.0,
 'State': 'Illinois',
 'Year': 2016,
 '_id': ObjectId('677707ae1f5535c6a24ea6d1'),
 'state_fips': '17'}


#### -One All Records For One State For One Year By Name (with .find)¶

In [179]:
query = {'$and': [
            {'State':'Illinois'},
            {'Year':2020},
        ]}
results = electionResults.find(query)
i = 0
for result in results: # pprint 3 records
    if i < 3:
        pprint(result)
    i+=1

{'Congressional_District': 'Congressional District 1',
 'District_dem_votes': 253802.0,
 'District_rep_votes': 97392.0,
 'State': 'Illinois',
 'Year': 2020,
 '_id': ObjectId('677707ae1f5535c6a24ea85b'),
 'state_fips': '17'}
{'Congressional_District': 'Congressional District 10',
 'District_dem_votes': 204803.0,
 'District_rep_votes': 109443.0,
 'State': 'Illinois',
 'Year': 2020,
 '_id': ObjectId('677707ae1f5535c6a24ea85c'),
 'state_fips': '17'}
{'Congressional_District': 'Congressional District 11',
 'District_dem_votes': 122111.0,
 'District_rep_votes': 100324.0,
 'State': 'Illinois',
 'Year': 2020,
 '_id': ObjectId('677707ae1f5535c6a24ea85d'),
 'state_fips': '17'}


#### - Total Votes For A State In One Election By Party (as list and as a dictionary)

In [182]:
query = {'$and': [
            {"State":'Illinois'},
            {"Year":2020},
        ]}
fields ={'District_dem_votes':1, 'District_rep_votes':1}
results = electionResults.find(query, fields)
result_list = list(results)

vote_count_list = [] # [total_dem_votes, total_rep_votes]
vote_count_dict = {}
total_dem_votes = 0
total_rep_votes = 0
for result in result_list:
    print(result)
    dem_votes = result['District_dem_votes']
    total_dem_votes += dem_votes
    
    rep_votes = result['District_rep_votes']
    total_rep_votes += rep_votes

total_dem_votes  = int(total_dem_votes)
total_rep_votes  = int(total_rep_votes)

vote_count_list.append(total_dem_votes)
vote_count_list.append(total_rep_votes)
vote_count_dict['total_dem_votes'] = total_dem_votes
vote_count_dict['total_rep_votes'] = total_rep_votes

{'_id': ObjectId('677707ae1f5535c6a24ea85b'), 'District_dem_votes': 253802.0, 'District_rep_votes': 97392.0}
{'_id': ObjectId('677707ae1f5535c6a24ea85c'), 'District_dem_votes': 204803.0, 'District_rep_votes': 109443.0}
{'_id': ObjectId('677707ae1f5535c6a24ea85d'), 'District_dem_votes': 122111.0, 'District_rep_votes': 100324.0}
{'_id': ObjectId('677707ae1f5535c6a24ea85e'), 'District_dem_votes': 140728.0, 'District_rep_votes': 190749.0}
{'_id': ObjectId('677707ae1f5535c6a24ea85f'), 'District_dem_votes': 152627.0, 'District_rep_votes': 191802.0}
{'_id': ObjectId('677707ae1f5535c6a24ea860'), 'District_dem_votes': 185260.0, 'District_rep_votes': 156345.0}
{'_id': ObjectId('677707ae1f5535c6a24ea861'), 'District_dem_votes': 93475.0, 'District_rep_votes': 233580.0}
{'_id': ObjectId('677707ae1f5535c6a24ea862'), 'District_dem_votes': 106088.0, 'District_rep_votes': 144799.0}
{'_id': ObjectId('677707ae1f5535c6a24ea863'), 'District_dem_votes': 149066.0, 'District_rep_votes': 169394.0}
{'_id': Obje

In [184]:
print(f"vote_count_list: {vote_count_list}")
print(f"vote_count_dict: {vote_count_dict}")

vote_count_list: [6290856, 4405582]
vote_count_dict: {'total_dem_votes': 6290856, 'total_rep_votes': 4405582}


#### - Total Votes For All States In One Election By Party (using 'aggregate(O)')

In [263]:
match_query = {'$match' : {'Year' : 2020}}
group_query = {'$group' : {'_id' :  '$State',
                           'total_dem_votes' : {'$sum': '$District_dem_votes'},
                           'total_rep_votes' : {'$sum': '$District_rep_votes'}}}
pipeline = [match_query,group_query]

In [265]:
results = electionResults.aggregate(pipeline)
results_list = list(results)
pprint(results_list[0:10])

[{'_id': 'Minnesota',
  'total_dem_votes': 3434134.0,
  'total_rep_votes': 2968110.0},
 {'_id': 'Iowa', 'total_dem_votes': 1131248.0, 'total_rep_votes': 1375628.0},
 {'_id': 'Oklahoma',
  'total_dem_votes': 1007772.0,
  'total_rep_votes': 2040552.0},
 {'_id': 'South Dakota',
  'total_dem_votes': 295284.0,
  'total_rep_votes': 521492.0},
 {'_id': 'Maine', 'total_dem_votes': 860944.0, 'total_rep_votes': 719796.0},
 {'_id': 'Idaho', 'total_dem_votes': 574040.0, 'total_rep_votes': 1108234.0},
 {'_id': 'North Carolina',
  'total_dem_votes': 5368558.0,
  'total_rep_votes': 5517520.0},
 {'_id': 'Nevada', 'total_dem_votes': 1406968.0, 'total_rep_votes': 1339776.0},
 {'_id': 'Louisiana',
  'total_dem_votes': 1699852.0,
  'total_rep_votes': 2480540.0},
 {'_id': 'Missouri',
  'total_dem_votes': 2504154.0,
  'total_rep_votes': 3429802.0}]


In [267]:
total_vote_df = pd.DataFrame(results_list)
total_vote_df.head()

,_id,total_dem_votes,total_rep_votes
0,Minnesota,3434134.0,2968110.0
1,Iowa,1131248.0,1375628.0
2,Oklahoma,1007772.0,2040552.0
3,South Dakota,295284.0,521492.0
4,Maine,860944.0,719796.0


#### - rename column '_id' -> 'States'

In [270]:
total_vote_df.rename(columns={'_id': 'States'}, inplace=True)
total_vote_df.head()

,States,total_dem_votes,total_rep_votes
0,Minnesota,3434134.0,2968110.0
1,Iowa,1131248.0,1375628.0
2,Oklahoma,1007772.0,2040552.0
3,South Dakota,295284.0,521492.0
4,Maine,860944.0,719796.0


#### - Get All State Name Abbreviations As Sorted List

In [409]:
query = {}
fields = {'state_abbrev':1}
sortSpec = [('state_abbrev',1)]
state_name_abbrevs = states.find(query, fields, sort = sortSpec)
#state_name_abbrevs_list = list(state_name_abbrevs)
state_name_abbrev_list = []
for result in state_name_abbrevs:
    state_name_abbrev = result['state_abbrev']
    state_name_abbrev_list.append(state_name_abbrev)
print(state_name_abbrev_list)

['AK', 'AK', 'AL', 'AL', 'AR', 'AR', 'AS', 'AS', 'AZ', 'AZ', 'CA', 'CA', 'CO', 'CO', 'CT', 'CT', 'DE', 'DE', 'FL', 'FL', 'GA', 'GA', 'GU', 'GU', 'HI', 'HI', 'IA', 'IA', 'ID', 'ID', 'IL', 'IL', 'IN', 'IN', 'KS', 'KS', 'KY', 'KY', 'LA', 'LA', 'MA', 'MA', 'MD', 'MD', 'ME', 'ME', 'MI', 'MI', 'MN', 'MN', 'MO', 'MO', 'MP', 'MP', 'MS', 'MS', 'MT', 'MT', 'NC', 'NC', 'ND', 'ND', 'NE', 'NE', 'NH', 'NH', 'NJ', 'NJ', 'NM', 'NM', 'NV', 'NV', 'NY', 'NY', 'OH', 'OH', 'OK', 'OK', 'OR', 'OR', 'PA', 'PA', 'PR', 'PR', 'RI', 'RI', 'SC', 'SC', 'SD', 'SD', 'TN', 'TN', 'TX', 'TX', 'UT', 'UT', 'VA', 'VA', 'VI', 'VI', 'VT', 'VT', 'WA', 'WA', 'WI', 'WI', 'WV', 'WV', 'WY', 'WY']


#### - Get All State Names <-> IDs, As Dictionary

In [412]:
query = {}
fields ={'state_name':1,'state_id':1}
results = states.find(query, fields)
result_list = list(results)
state_name_id_dict = {}
for result in result_list:
    state_id = result['state_id']
    state_name = result['state_name']
    state_name_id_dict[state_name] = state_id 
print(state_name_id_dict)

{'Arizona': 2, 'Connecticut': 6, 'Colorado': 5, 'Florida': 8, 'California': 4, 'Georgia': 9, 'Hawaii': 10, 'Idaho': 11, 'Indiana': 13, 'Iowa': 14, 'Kansas': 15, 'Louisiana': 17, 'Maine': 18, 'Kentucky': 16, 'Maryland': 19, 'Massachusetts': 20, 'Arkansas': 3, 'Delaware': 7, 'Alaska': 1, 'Illinois': 12, 'Alabama': 0, 'Michigan': 21, 'Mississippi': 23, 'Minnesota': 22, 'Montana': 25, 'Nebraska': 26, 'Nevada': 27, 'New Jersey': 29, 'New Hampshire': 28, 'Oregon': 36, 'Pennsylvania': 37, 'Missouri': 24, 'Rhode Island': 38, 'Oklahoma': 35, 'North Carolina': 32, 'South Carolina': 39, 'South Dakota': 40, 'Tennessee': 41, 'Texas': 42, 'Utah': 43, 'Virginia': 45, 'New Mexico': 30, 'Washington': 46, 'West Virginia': 47, 'Wisconsin': 48, 'Vermont': 44, 'American Samoa': 50, 'Guam': 51, 'Northern Mariana Islands': 52, 'Puerto Rico': 53, 'Virgin Islands': 54, 'Wyoming': 49, 'North Dakota': 33, 'Ohio': 34, 'New York': 31}


#### - Get All State Names <-> Fips, As Dictioary

In [415]:
query = {}
fields ={'state_name':1,'fips':1}
results = states.find(query, fields)
result_list = list(results)
state_name_fips_dict = {}
for result in result_list:
    fips = result['fips']
    state_name = result['state_name']
    state_name_fips_dict[state_name] = fips 
print(state_name_fips_dict)

{'Arizona': '04', 'Connecticut': '09', 'Colorado': '08', 'Florida': '12', 'California': '06', 'Georgia': '13', 'Hawaii': '15', 'Idaho': '16', 'Indiana': '18', 'Iowa': '19', 'Kansas': '20', 'Louisiana': '22', 'Maine': '23', 'Kentucky': '21', 'Maryland': '24', 'Massachusetts': '25', 'Arkansas': '05', 'Delaware': '10', 'Alaska': '02', 'Illinois': '17', 'Alabama': '01', 'Michigan': '26', 'Mississippi': '28', 'Minnesota': '27', 'Montana': '30', 'Nebraska': '31', 'Nevada': '32', 'New Jersey': '34', 'New Hampshire': '33', 'Oregon': '41', 'Pennsylvania': '42', 'Missouri': '29', 'Rhode Island': '44', 'Oklahoma': '40', 'North Carolina': '37', 'South Carolina': '45', 'South Dakota': '46', 'Tennessee': '47', 'Texas': '48', 'Utah': '49', 'Virginia': '51', 'New Mexico': '35', 'Washington': '53', 'West Virginia': '54', 'Wisconsin': '55', 'Vermont': '50', 'American Samoa': '60', 'Guam': '66', 'Northern Mariana Islands': '69', 'Puerto Rico': '72', 'Virgin Islands': '78', 'Wyoming': '56', 'North Dakota'

#### - Move Database into Dataframe

In [418]:
allData = list(states.find())
df = pd.DataFrame(allData)
df.head()

,_id,state_id,state_name,state_abbrev,fips
0,6776c262f0920008d07495ae,2,Arizona,AZ,04
1,6776c262f0920008d07495af,6,Connecticut,CT,09
2,6776c262f0920008d07495b0,5,Colorado,CO,08
3,6776c262f0920008d07495b1,8,Florida,FL,12
4,6776c262f0920008d07495b2,4,California,CA,06
